In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import random
from multiprocessing import Pool

def convert_annotation_file(vgg_file_path, yolo_file_path, img_size):
    with open(vgg_file_path, 'r') as f:
        lines = f.readlines()

    img_path = lines[0].strip()
    img = cv2.imread(img_path)
    height, width, channels = img.shape

    yolo_lines = []
    for line in lines[1:]:
        data = line.strip().split()
        class_name = data[0]
        x_min, y_min, x_max, y_max = list(map(float, data[1:]))

        x_center = (x_min + x_max) / 2 / width
        y_center = (y_min + y_max) / 2 / height
        box_width = (x_max - x_min) / width
        box_height = (y_max - y_min) / height

        yolo_lines.append(f"{class_name} {x_center} {y_center} {box_width} {box_height}\n")

    with open(yolo_file_path, 'w') as f:
        f.writelines(yolo_lines)


def augment_image(img_path, save_dir, img_size):
    img = Image.open(img_path)
    img_name, ext = os.path.splitext(os.path.basename(img_path))

    
    brightness_levels = [-70, -35, 0, 35, 70]
    for i, level in enumerate(brightness_levels):
        new_img = np.array(img)
        new_img = new_img + level
        new_img = np.clip(new_img, 0, 255)
        new_img = Image.fromarray(new_img.astype(np.uint8))
        
        
        new_img = new_img.resize((img_size, img_size))
        
        new_img.save(os.path.join(save_dir, f"{img_name}_brightness_{i+1}{ext}"))

    
    noise_levels = [10, 20, 30, 40, 50]
    for i, level in enumerate(noise_levels):
        new_img = np.array(img)
        noise = np.random.normal(0, level, size=new_img.shape)
        new_img = new_img + noise
        new_img = np.clip(new_img, 0, 255)
        new_img = Image.fromarray(new_img.astype(np.uint8))
        
        # Resize image
        new_img = new_img.resize((img_size, img_size))
        
        new_img.save(os.path.join(save_dir, f"{img_name}_noise_{i+1}{ext}"))


if __name__ == '__main__':
    
    vgg_file_path = 'annotation/file.txt'
    yolo_file_path = 'annotation/file.txt'
    img_dir = 'directory'
    save_dir = 'augmented/images'
    img_size = 416  # 
    
    
    convert_annotation_file(vgg_file_path, yolo_file_path, img_size)

    
    img_paths = [os.path.join(img_dir, img_name) for img_name in os.listdir(img_dir)]
    with Pool() as pool:
        pool.starmap(augment_image, [(img_path, save_dir, img_size) for img_path in img_paths])